# Part 2: License Plate Bounding Box Detection.

**Goal**: Build a model that finds the exact location coordinates of license plates in images


## **What We'll Build (Actually Interesting!):**
1. ** Bounding Box Detector**: Predicts (x, y, width, height) coordinates
2. ** Real Object Detection**: Not just "is there a plate?" but "WHERE is the plate?"
3. ** Regression Model**: Outputs actual coordinate values
4. ** Visual Results**: See detected boxes overlaid on images
5. ** Production-Ready**: Downloadable detection model


## **Technical Challenge:**
- **Input**: 400x400 RGB image
- **Output**: [x_center, y_center, width, height] normalized coordinates
- **Loss**: Mean Squared Error for coordinate regression
- **Evaluation**: IoU (Intersection over Union) with ground truth


## **Setup and Imports**

In [1]:
# Essential imports for bounding box detection
import os
import xml.etree.ElementTree as ET
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import cv2
import random
from pathlib import Path
import zipfile
import json
from datetime import datetime

# Deep learning for regression
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

print(" PART 2: LICENSE PLATE BOUNDING BOX DETECTION")
print("=" * 55)
print(f" TensorFlow version: {tf.__version__}")
print(f"  GPU available: {len(tf.config.list_physical_devices('GPU')) > 0}")
print(" Building REAL object detection (not boring classification)!")
print(" Setup complete!")


2025-07-18 16:09:00.657157: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-18 16:09:01.072353: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-18 16:09:01.176880: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752854941.425817      90 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752854941.457904      90 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752854941.767435      90 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

 PART 2: LICENSE PLATE BOUNDING BOX DETECTION
 TensorFlow version: 2.19.0
  GPU available: False
 Building REAL object detection (not boring classification)!
 Setup complete!


2025-07-18 16:09:30.959194: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


## **Load Dataset and Extract Bounding Boxes**

In [2]:
# Dataset configuration
DATA_DIR = "lpr_data"
IMAGES_DIR = os.path.join(DATA_DIR, "images")
ANNOTATIONS_DIR = os.path.join(DATA_DIR, "annotations")
MODEL_SAVE_DIR = "saved_models"
IMG_SIZE = 400  # Input size for model

# Create directories
os.makedirs(MODEL_SAVE_DIR, exist_ok=True)

def parse_bounding_box(xml_path):
    """Extract bounding box coordinates from XML annotation"""
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()
        
        # Get image dimensions
        size = root.find('size')
        img_width = int(size.find('width').text)
        img_height = int(size.find('height').text)
        
        # Find license plate bounding boxes
        bboxes = []
        for obj in root.findall('object'):
            name_elem = obj.find('name')
            n_elem = obj.find('n')
            
            obj_name = ""
            if name_elem is not None and name_elem.text:
                obj_name = name_elem.text.lower()
            elif n_elem is not None and n_elem.text:
                obj_name = n_elem.text.lower()
            
            # Check if it's a license plate
            if 'licen' in obj_name or 'plate' in obj_name:
                bbox = obj.find('bndbox')
                if bbox is not None:
                    xmin = int(bbox.find('xmin').text)
                    ymin = int(bbox.find('ymin').text)
                    xmax = int(bbox.find('xmax').text)
                    ymax = int(bbox.find('ymax').text)
                    
                    # Convert to center coordinates and normalize
                    x_center = (xmin + xmax) / 2.0 / img_width
                    y_center = (ymin + ymax) / 2.0 / img_height
                    width = (xmax - xmin) / img_width
                    height = (ymax - ymin) / img_height
                    
                    bboxes.append([x_center, y_center, width, height])
        
        return bboxes, img_width, img_height
        
    except Exception as e:
        print(f" Error parsing {xml_path}: {e}")
        return [], 0, 0

def load_and_preprocess_image(image_path, target_size=IMG_SIZE):
    """Load and resize image while maintaining aspect ratio"""
    try:
        image = cv2.imread(image_path)
        if image is None:
            return None
            
        # Convert BGR to RGB
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        # Resize image
        image = cv2.resize(image, (target_size, target_size))
        
        # Normalize to [0, 1]
        image = image.astype(np.float32) / 255.0
        
        return image
        
    except Exception as e:
        print(f" Error loading {image_path}: {e}")
        return None

print(" LOADING BOUNDING BOX DATASET")
print("=" * 35)

# Check dataset
if not os.path.exists(DATA_DIR):
    print(f" Dataset not found: {DATA_DIR}")
    print("📥 Please ensure you have the lpr_data directory")
else:
    image_files = [f for f in os.listdir(IMAGES_DIR) if f.endswith('.png')] if os.path.exists(IMAGES_DIR) else []
    annotation_files = [f for f in os.listdir(ANNOTATIONS_DIR) if f.endswith('.xml')] if os.path.exists(ANNOTATIONS_DIR) else []
    
    print(f" Dataset found!")
    print(f"     Images: {len(image_files)}")
    print(f"    Annotations: {len(annotation_files)}")
    
    # Process first few files to show what we're doing
    if len(image_files) > 0:
        print(f"\n Sample bounding box extraction:")
        for i in range(min(3, len(image_files))):
            xml_file = image_files[i].replace('.png', '.xml')
            xml_path = os.path.join(ANNOTATIONS_DIR, xml_file)
            
            if os.path.exists(xml_path):
                bboxes, w, h = parse_bounding_box(xml_path)
                print(f"    {image_files[i]} ({w}x{h}): {len(bboxes)} license plates")
                for j, bbox in enumerate(bboxes):
                    print(f"       Plate {j+1}: center=({bbox[0]:.3f}, {bbox[1]:.3f}), size=({bbox[2]:.3f}, {bbox[3]:.3f})")


 LOADING BOUNDING BOX DATASET
 Dataset found!
     Images: 433
    Annotations: 433

 Sample bounding box extraction:
    Cars112.png (240x400): 1 license plates
       Plate 1: center=(0.444, 0.407), size=(0.479, 0.140)
    Cars310.png (600x531): 1 license plates
       Plate 1: center=(0.491, 0.740), size=(0.178, 0.090)
    Cars31.png (400x245): 1 license plates
       Plate 1: center=(0.858, 0.784), size=(0.285, 0.155)


##  **Prepare Training Data for Bounding Box Regression**

In [3]:
print(" PREPARING BOUNDING BOX TRAINING DATA")
print("=" * 40)

# Prepare training data
X_data = []  # Images
y_data = []  # Bounding box coordinates [x_center, y_center, width, height]

if len(image_files) > 0:
    print(f" Processing {len(image_files)} images for bounding box training...")
    
    valid_samples = 0
    multiple_plates = 0
    
    for i, img_file in enumerate(image_files):
        if i % 100 == 0:
            print(f"    Processed {i}/{len(image_files)} images...")
        
        # Load image
        img_path = os.path.join(IMAGES_DIR, img_file)
        image = load_and_preprocess_image(img_path)
        if image is None:
            continue
        
        # Load bounding box
        xml_file = img_file.replace('.png', '.xml')
        xml_path = os.path.join(ANNOTATIONS_DIR, xml_file)
        
        if not os.path.exists(xml_path):
            continue
            
        bboxes, orig_w, orig_h = parse_bounding_box(xml_path)
        
        if len(bboxes) == 0:
            continue
        elif len(bboxes) > 1:
            multiple_plates += 1
            # For simplicity, use the first bounding box (largest usually)
            # In production, you'd use multi-object detection
            bbox = bboxes[0]
        else:
            bbox = bboxes[0]
        
        # Add to training data
        X_data.append(image)
        y_data.append(bbox)  # [x_center, y_center, width, height] normalized
        valid_samples += 1
    
    # Convert to numpy arrays
    X_data = np.array(X_data)
    y_data = np.array(y_data)
    
    print(f"\n Training data prepared!")
    print(f"    Total valid samples: {len(X_data)}")
    print(f"    Image shape: {X_data[0].shape}")
    print(f"    Bbox shape: {y_data[0].shape}")
    print(f"    Images with multiple plates: {multiple_plates}")
    print(f"    Sample bounding boxes:")
    for i in range(min(5, len(y_data))):
        x, y, w, h = y_data[i]
        print(f"       Sample {i+1}: center=({x:.3f}, {y:.3f}), size=({w:.3f}, {h:.3f})")
    
    # Verify data quality
    print(f"\n🔍 Data Quality Check:")
    print(f"    X coordinate range: {y_data[:, 0].min():.3f} - {y_data[:, 0].max():.3f}")
    print(f"    Y coordinate range: {y_data[:, 1].min():.3f} - {y_data[:, 1].max():.3f}")
    print(f"    Width range: {y_data[:, 2].min():.3f} - {y_data[:, 2].max():.3f}")
    print(f"    Height range: {y_data[:, 3].min():.3f} - {y_data[:, 3].max():.3f}")
    
else:
    print(" No images found!")
    print(" Creating dummy data for demonstration...")
    # Create synthetic data for demo
    X_data = np.random.random((50, IMG_SIZE, IMG_SIZE, 3)).astype(np.float32)
    y_data = np.random.random((50, 4)).astype(np.float32)  # Random normalized coordinates
    print(f" Dummy data created: {len(X_data)} samples")


 PREPARING BOUNDING BOX TRAINING DATA
 Processing 433 images for bounding box training...
    Processed 0/433 images...
    Processed 100/433 images...
    Processed 200/433 images...
    Processed 300/433 images...
    Processed 400/433 images...

 Training data prepared!
    Total valid samples: 433
    Image shape: (400, 400, 3)
    Bbox shape: (4,)
    Images with multiple plates: 24
    Sample bounding boxes:
       Sample 1: center=(0.444, 0.407), size=(0.479, 0.140)
       Sample 2: center=(0.491, 0.740), size=(0.178, 0.090)
       Sample 3: center=(0.858, 0.784), size=(0.285, 0.155)
       Sample 4: center=(0.249, 0.750), size=(0.242, 0.144)
       Sample 5: center=(0.321, 0.468), size=(0.188, 0.112)

🔍 Data Quality Check:
    X coordinate range: 0.034 - 0.949
    Y coordinate range: 0.202 - 0.979
    Width range: 0.022 - 0.972
    Height range: 0.020 - 0.800


##  **Build Bounding Box Regression Model**

In [4]:
def create_bbox_detector():
    """Create a CNN model for bounding box regression"""
    
    model = keras.Sequential([
        # Input layer
        layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3)),
        
        # Feature extraction backbone (similar to VGG-style)
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        layers.Conv2D(512, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Regression head
        layers.GlobalAveragePooling2D(),
        layers.Dense(1024, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(4, activation='sigmoid')  # Output: [x_center, y_center, width, height]
    ])
    
    return model

def iou_metric(y_true, y_pred):
    """Calculate IoU (Intersection over Union) metric for bounding boxes"""
    # Convert from center coordinates to corner coordinates
    def center_to_corner(boxes):
        x_center, y_center, width, height = tf.split(boxes, 4, axis=-1)
        x1 = x_center - width / 2
        y1 = y_center - height / 2
        x2 = x_center + width / 2
        y2 = y_center + height / 2
        return tf.concat([x1, y1, x2, y2], axis=-1)
    
    true_corners = center_to_corner(y_true)
    pred_corners = center_to_corner(y_pred)
    
    # Calculate intersection
    x1 = tf.maximum(true_corners[..., 0], pred_corners[..., 0])
    y1 = tf.maximum(true_corners[..., 1], pred_corners[..., 1])
    x2 = tf.minimum(true_corners[..., 2], pred_corners[..., 2])
    y2 = tf.minimum(true_corners[..., 3], pred_corners[..., 3])
    
    intersection = tf.maximum(0.0, x2 - x1) * tf.maximum(0.0, y2 - y1)
    
    # Calculate union
    true_area = (true_corners[..., 2] - true_corners[..., 0]) * (true_corners[..., 3] - true_corners[..., 1])
    pred_area = (pred_corners[..., 2] - pred_corners[..., 0]) * (pred_corners[..., 3] - pred_corners[..., 1])
    union = true_area + pred_area - intersection
    
    # Calculate IoU
    iou = intersection / (union + 1e-7)
    return tf.reduce_mean(iou)

print(" BUILDING BOUNDING BOX REGRESSION MODEL")
print("=" * 45)

# Create the model
model = create_bbox_detector()

# Compile with appropriate loss and metrics for regression
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='mse',  # Mean Squared Error for coordinate regression
    metrics=[iou_metric, 'mae']  # IoU and Mean Absolute Error
)

# Display model summary
print(" Model Architecture:")
model.summary()

print(f"\n Model Statistics:")
print(f"    Total parameters: {model.count_params():,}")
print(f"    Model type: Bounding Box Regression CNN")
print(f"    Task: Predict [x_center, y_center, width, height]")
print(f"    Input: {IMG_SIZE}x{IMG_SIZE}x3 RGB images")
print(f"    Output: 4 normalized coordinates [0,1]")
print(f"    Loss: MSE (Mean Squared Error)")
print(f"    Metrics: IoU (Intersection over Union) + MAE")


 BUILDING BOUNDING BOX REGRESSION MODEL
 Model Architecture:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 400, 400, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 400, 400, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 200, 200, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 200, 200, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 200, 200, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 200, 200, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 100, 100, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 100, 100, 128)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 100, 100, 256)  │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 100, 100, 256)  │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 50, 50, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 50, 50, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 50, 50, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 50, 50, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 25, 25, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 25, 25, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 512)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1024)           │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 4)              │         2,052 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,737,540 (21.89 MB)

 Trainable params: 5,737,540 (21.89 MB)

 Non-trainable params: 0 (0.00 B)


 Model Statistics:
    Total parameters: 5,737,540
    Model type: Bounding Box Regression CNN
    Task: Predict [x_center, y_center, width, height]
    Input: 400x400x3 RGB images
    Output: 4 normalized coordinates [0,1]
    Loss: MSE (Mean Squared Error)
    Metrics: IoU (Intersection over Union) + MAE


##  **Train the Bounding Box Model**


In [ ]:
print(" TRAINING BOUNDING BOX DETECTION MODEL")
print("=" * 45)

if len(X_data) > 10:  # Need sufficient data for training
    # Split data
    X_train, X_val, y_train, y_val = train_test_split(
        X_data, y_data, 
        test_size=0.2, 
        random_state=42
    )
    
    print(f" Data Split:")
    print(f"    Training samples: {len(X_train)}")
    print(f"    Validation samples: {len(X_val)}")
    
    # Verify coordinate ranges
    print(f"\n Training coordinate ranges:")
    print(f"    X: {y_train[:, 0].min():.3f} - {y_train[:, 0].max():.3f}")
    print(f"    Y: {y_train[:, 1].min():.3f} - {y_train[:, 1].max():.3f}")
    print(f"    W: {y_train[:, 2].min():.3f} - {y_train[:, 2].max():.3f}")
    print(f"    H: {y_train[:, 3].min():.3f} - {y_train[:, 3].max():.3f}")
    
    # Train the model
    print(f"\n Starting bounding box regression training...")
    
    # Create callbacks for better training
    callbacks = [
        keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=5,
            restore_best_weights=True
        ),
        keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=3,
            min_lr=1e-7
        )
    ]
    
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=2,  # More epochs for regression
        batch_size=16,  # Smaller batch size for better gradients
        callbacks=callbacks,
        verbose=1
    )
    
    # Get final metrics
    final_train_loss = history.history['loss'][-1]
    final_val_loss = history.history['val_loss'][-1]
    final_train_iou = history.history['iou_metric'][-1]
    final_val_iou = history.history['val_iou_metric'][-1]
    final_train_mae = history.history['mae'][-1]
    final_val_mae = history.history['val_mae'][-1]
    
    print(f"\n TRAINING COMPLETE!")
    print(f"    Final Training Loss (MSE): {final_train_loss:.4f}")
    print(f"    Final Validation Loss (MSE): {final_val_loss:.4f}")
    print(f"    Final Training IoU: {final_train_iou:.4f}")
    print(f"    Final Validation IoU: {final_val_iou:.4f}")
    print(f"    Final Training MAE: {final_train_mae:.4f}")
    print(f"    Final Validation MAE: {final_val_mae:.4f}")
    
    # Plot training history
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
    
    # Loss plot
    ax1.plot(history.history['loss'], 'b-', label='Training Loss', linewidth=2)
    ax1.plot(history.history['val_loss'], 'r-', label='Validation Loss', linewidth=2)
    ax1.set_title(' Model Loss (MSE)', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Epoch')
    ax1.set_ylabel('Mean Squared Error')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # IoU plot
    ax2.plot(history.history['iou_metric'], 'b-', label='Training IoU', linewidth=2)
    ax2.plot(history.history['val_iou_metric'], 'r-', label='Validation IoU', linewidth=2)
    ax2.set_title(' IoU Score', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Epoch')
    ax2.set_ylabel('Intersection over Union')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # MAE plot
    ax3.plot(history.history['mae'], 'b-', label='Training MAE', linewidth=2)
    ax3.plot(history.history['val_mae'], 'r-', label='Validation MAE', linewidth=2)
    ax3.set_title(' Mean Absolute Error', fontsize=14, fontweight='bold')
    ax3.set_xlabel('Epoch')
    ax3.set_ylabel('MAE')
    ax3.legend()
    ax3.grid(True, alpha=0.3)
    
    # Learning rate plot (if available)
    if 'lr' in history.history:
        ax4.plot(history.history['lr'], 'g-', linewidth=2)
        ax4.set_title(' Learning Rate', fontsize=14, fontweight='bold')
        ax4.set_xlabel('Epoch')
        ax4.set_ylabel('Learning Rate')
        ax4.set_yscale('log')
        ax4.grid(True, alpha=0.3)
    else:
        ax4.text(0.5, 0.5, 'Learning Rate\\nNot Tracked', ha='center', va='center', 
                transform=ax4.transAxes, fontsize=12)
        ax4.set_title(' Learning Rate', fontsize=14, fontweight='bold')
    
    plt.tight_layout()
    plt.show()
    
    # Store training results
    training_results = {
        'final_train_loss': float(final_train_loss),
        'final_val_loss': float(final_val_loss),
        'final_train_iou': float(final_train_iou),
        'final_val_iou': float(final_val_iou),
        'final_train_mae': float(final_train_mae),
        'final_val_mae': float(final_val_mae),
        'epochs_trained': len(history.history['loss']),
        'train_samples': len(X_train),
        'val_samples': len(X_val)
    }
    
else:
    print(" Insufficient data for training!")
    print(f"    Current samples: {len(X_data)}")
    print("    Need at least 10 samples for meaningful training")
    training_results = {'status': 'insufficient_data'}


 TRAINING BOUNDING BOX DETECTION MODEL
 Data Split:
    Training samples: 346
    Validation samples: 87

 Training coordinate ranges:
    X: 0.060 - 0.949
    Y: 0.202 - 0.979
    W: 0.022 - 0.972
    H: 0.020 - 0.800

 Starting bounding box regression training...
Epoch 1/2
 1/22 ━━━━━━━━━━━━━━━━━━━━ 3:32 10s/step - iou_metric: 0.0896 - loss: 0.0771 - mae: 0.2381

##  **Visualize Predictions vs Ground Truth**

In [ ]:
def visualize_bbox_predictions(images, true_bboxes, pred_bboxes, num_samples=6):
    """Visualize predicted vs ground truth bounding boxes"""
    
    num_samples = min(num_samples, len(images))
    cols = 3
    rows = (num_samples + cols - 1) // cols
    
    fig, axes = plt.subplots(rows, cols, figsize=(15, 5 * rows))
    if rows == 1:
        axes = axes.reshape(1, -1)
    
    for i in range(num_samples):
        row = i // cols
        col = i % cols
        ax = axes[row, col]
        
        # Display image
        ax.imshow(images[i])
        
        # Convert normalized coordinates to pixel coordinates
        img_h, img_w = images[i].shape[:2]
        
        # Ground truth box (green)
        true_bbox = true_bboxes[i]
        true_x_center, true_y_center, true_width, true_height = true_bbox
        true_x1 = (true_x_center - true_width / 2) * img_w
        true_y1 = (true_y_center - true_height / 2) * img_h
        true_w = true_width * img_w
        true_h = true_height * img_h
        
        true_rect = patches.Rectangle(
            (true_x1, true_y1), true_w, true_h,
            linewidth=3, edgecolor='green', facecolor='none',
            label='Ground Truth'
        )
        ax.add_patch(true_rect)
        
        # Predicted box (red)
        pred_bbox = pred_bboxes[i]
        pred_x_center, pred_y_center, pred_width, pred_height = pred_bbox
        pred_x1 = (pred_x_center - pred_width / 2) * img_w
        pred_y1 = (pred_y_center - pred_height / 2) * img_h
        pred_w = pred_width * img_w
        pred_h = pred_height * img_h
        
        pred_rect = patches.Rectangle(
            (pred_x1, pred_y1), pred_w, pred_h,
            linewidth=3, edgecolor='red', facecolor='none',
            label='Prediction'
        )
        ax.add_patch(pred_rect)
        
        # Calculate IoU for this sample
        def calculate_iou_single(box1, box2):
            # Convert to corner coordinates
            x1_1 = box1[0] - box1[2] / 2
            y1_1 = box1[1] - box1[3] / 2
            x2_1 = box1[0] + box1[2] / 2
            y2_1 = box1[1] + box1[3] / 2
            
            x1_2 = box2[0] - box2[2] / 2
            y1_2 = box2[1] - box2[3] / 2
            x2_2 = box2[0] + box2[2] / 2
            y2_2 = box2[1] + box2[3] / 2
            
            # Calculate intersection
            x1_i = max(x1_1, x1_2)
            y1_i = max(y1_1, y1_2)
            x2_i = min(x2_1, x2_2)
            y2_i = min(y2_1, y2_2)
            
            if x2_i <= x1_i or y2_i <= y1_i:
                return 0.0
            
            intersection = (x2_i - x1_i) * (y2_i - y1_i)
            area1 = box1[2] * box1[3]
            area2 = box2[2] * box2[3]
            union = area1 + area2 - intersection
            
            return intersection / union if union > 0 else 0.0
        
        iou = calculate_iou_single(true_bbox, pred_bbox)
        
        ax.set_title(f'Sample {i+1}\\nIoU: {iou:.3f}', fontsize=12, fontweight='bold')
        ax.axis('off')
        
        # Add legend to first subplot
        if i == 0:
            ax.legend(loc='upper right')
    
    # Hide empty subplots
    for i in range(num_samples, rows * cols):
        row = i // cols
        col = i % cols
        axes[row, col].axis('off')
    
    plt.tight_layout()
    plt.show()

print(" VISUALIZING BOUNDING BOX PREDICTIONS")
print("=" * 40)

if len(X_data) > 10 and 'X_val' in locals():
    # Get predictions on validation set
    print(" Getting predictions on validation set...")
    
    val_predictions = model.predict(X_val, verbose=0)
    
    # Calculate IoU for all validation samples
    ious = []
    for i in range(len(X_val)):
        true_bbox = y_val[i]
        pred_bbox = val_predictions[i]
        
        # Calculate IoU
        def calculate_iou_single(box1, box2):
            x1_1 = box1[0] - box1[2] / 2
            y1_1 = box1[1] - box1[3] / 2
            x2_1 = box1[0] + box1[2] / 2
            y2_1 = box1[1] + box1[3] / 2
            
            x1_2 = box2[0] - box2[2] / 2
            y1_2 = box2[1] - box2[3] / 2
            x2_2 = box2[0] + box2[2] / 2
            y2_2 = box2[1] + box2[3] / 2
            
            x1_i = max(x1_1, x1_2)
            y1_i = max(y1_1, y1_2)
            x2_i = min(x2_1, x2_2)
            y2_i = min(y2_1, y2_2)
            
            if x2_i <= x1_i or y2_i <= y1_i:
                return 0.0
            
            intersection = (x2_i - x1_i) * (y2_i - y1_i)
            area1 = box1[2] * box1[3]
            area2 = box2[2] * box2[3]
            union = area1 + area2 - intersection
            
            return intersection / union if union > 0 else 0.0
        
        iou = calculate_iou_single(true_bbox, pred_bbox)
        ious.append(iou)
    
    mean_iou = np.mean(ious)
    print(f" Validation Set Performance:")
    print(f"    Mean IoU: {mean_iou:.4f}")
    print(f"    Best IoU: {max(ious):.4f}")
    print(f"    Worst IoU: {min(ious):.4f}")
    print(f"    IoU > 0.5: {sum(1 for iou in ious if iou > 0.5)}/{len(ious)} ({100*sum(1 for iou in ious if iou > 0.5)/len(ious):.1f}%)")
    
    # Visualize best and worst predictions
    print(f"\n Showing validation predictions:")
    
    # Sort by IoU to show best and worst
    sorted_indices = np.argsort(ious)
    
    # Show a mix of best and worst predictions
    display_indices = []
    display_indices.extend(sorted_indices[-3:])  # Best 3
    display_indices.extend(sorted_indices[:3])   # Worst 3
    
    display_images = X_val[display_indices]
    display_true = y_val[display_indices]
    display_pred = val_predictions[display_indices]
    
    visualize_bbox_predictions(display_images, display_true, display_pred, 6)
    
    # Update training results with IoU
    training_results['mean_validation_iou'] = float(mean_iou)
    training_results['best_validation_iou'] = float(max(ious))
    training_results['worst_validation_iou'] = float(min(ious))
    
else:
    print(" No validation data available for visualization")
    print("   Creating demo visualization...")
    
    # Create dummy visualization
    demo_images = np.random.random((3, 400, 400, 3))
    demo_true = np.array([[0.5, 0.5, 0.3, 0.2], [0.6, 0.4, 0.25, 0.15], [0.4, 0.6, 0.35, 0.25]])
    demo_pred = demo_true + np.random.normal(0, 0.05, demo_true.shape)
    demo_pred = np.clip(demo_pred, 0, 1)  # Keep in valid range
    
    visualize_bbox_predictions(demo_images, demo_true, demo_pred, 3)
    print(" Demo visualization complete!")


## **Save and Download Bounding Box Model**

In [ ]:
print(" SAVING BOUNDING BOX DETECTION MODEL")
print("=" * 40)

# Create model name with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
model_name = f"bbox_detector_{timestamp}"
model_path = os.path.join(MODEL_SAVE_DIR, model_name)

print(f" Saving model: {model_name}")

# Save in multiple formats
try:
    # 1. Save as .keras format (Keras 3 native format - recommended)
    keras_model_path = f"{model_path}.keras"
    model.save(keras_model_path)
    print(f"   ✅ Keras format: {keras_model_path}")
    
    # 2. Export as SavedModel format (for TensorFlow Serving/TFLite)
    savedmodel_path = f"{model_path}_savedmodel"
    try:
        model.export(savedmodel_path)
        print(f"   ✅ SavedModel format: {savedmodel_path}")
    except Exception as e:
        print(f"   ⚠️  SavedModel export failed: {e}")
        # Fallback: try using tf.saved_model.save
        try:
            tf.saved_model.save(model, savedmodel_path)
            print(f"   ✅ SavedModel format (fallback): {savedmodel_path}")
        except Exception as e2:
            print(f"   ❌ SavedModel save failed: {e2}")

    # 3. Save as H5 format (for backward compatibility)
    h5_model_path = f"{model_path}.h5"
    model.save(h5_model_path)
    print(f"   ✅ H5 format: {h5_model_path}")

    # 4. Save model weights only (Keras 3 compatible)
    weights_path = f"{model_path}.weights.h5"
    model.save_weights(weights_path)
    print(f"   ✅ Weights: {weights_path}")
    
    # 4. Architecture JSON
    architecture_path = f"{model_path}_architecture.json"
    with open(architecture_path, 'w') as f:
        f.write(model.to_json())
    print(f"   ✅ Architecture: {architecture_path}")
    
    # 5. Model metadata
    metadata = {
        'model_name': model_name,
        'created_at': datetime.now().isoformat(),
        'tensorflow_version': tf.__version__,
        'model_type': 'Bounding Box Regression CNN',
        'task': 'License Plate Detection',
        'input_shape': [IMG_SIZE, IMG_SIZE, 3],
        'output_shape': [4],
        'output_description': '[x_center, y_center, width, height] normalized',
        'parameters': int(model.count_params()),
        'training_results': training_results
    }
    
    metadata_path = f"{model_path}_metadata.json"
    with open(metadata_path, 'w') as f:
        json.dump(metadata, f, indent=2)
    print(f"   ✅ Metadata: {metadata_path}")
    zipf.writestr(f"{model_name}_README.md", readme_content)
    
    # Get file sizes
    package_size = os.path.getsize(package_path) / (1024 * 1024)
    keras_size = os.path.getsize(keras_model_path) / (1024 * 1024)
    h5_size = os.path.getsize(h5_model_path) / (1024 * 1024)
    
    print(f"\n📦 MODEL PACKAGE CREATED!")
    print(f"   📁 Package: {model_name}")
    print(f"   📊 Keras size: {keras_size:.2f} MB")
    print(f"   📊 H5 size: {h5_size:.2f} MB")
    print(f"   📦 Package size: {package_size:.2f} MB")
    print(f"   🔢 Parameters: {model.count_params():,}")
    
    print(f"\n🎯 DOWNLOAD READY!")
    print(f"   📁 File: {package_path}")
    print(f"   🚀 Load with: tf.keras.models.load_model('{model_name}.keras')")
    print(f"   💡 Alternative: tf.keras.models.load_model('{model_name}.h5')")
    
except Exception as e:
    print(f"❌ Error saving model: {e}")
    print("   Model training completed but saving failed")



In [ ]:
# Inference code:

In [ ]:
    
    # 6. Create inference example
    inference_code = f'''# License Plate Bounding Box Detection - Inference Example
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Load the model
model = tf.keras.models.load_model('{model_name}.h5')

def predict_license_plate(image_path):
    """Predict license plate bounding box in an image"""
    
    # Load and preprocess image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    original_shape = image.shape[:2]
    
    # Resize to model input size
    image_resized = cv2.resize(image, ({IMG_SIZE}, {IMG_SIZE}))
    image_normalized = image_resized.astype(np.float32) / 255.0
    
    # Add batch dimension and predict
    image_batch = np.expand_dims(image_normalized, axis=0)
    prediction = model.predict(image_batch)[0]
    
    # Convert normalized coordinates to original image coordinates
    x_center, y_center, width, height = prediction
    
    # Convert to pixel coordinates
    orig_h, orig_w = original_shape
    x_center_px = x_center * orig_w
    y_center_px = y_center * orig_h
    width_px = width * orig_w
    height_px = height * orig_h
    
    # Convert to corner coordinates
    x1 = x_center_px - width_px / 2
    y1 = y_center_px - height_px / 2
    x2 = x_center_px + width_px / 2
    y2 = y_center_px + height_px / 2
    
    return {{
        'bbox_normalized': prediction,
        'bbox_pixels': [x1, y1, x2, y2],
        'center': [x_center_px, y_center_px],
        'size': [width_px, height_px]
    }}

def visualize_prediction(image_path):
    """Visualize license plate detection result"""
    
    # Load original image
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Get prediction
    result = predict_license_plate(image_path)
    x1, y1, x2, y2 = result['bbox_pixels']
    
    # Visualize
    fig, ax = plt.subplots(1, 1, figsize=(12, 8))
    ax.imshow(image)
    
    # Draw bounding box
    rect = patches.Rectangle(
        (x1, y1), x2-x1, y2-y1,
        linewidth=3, edgecolor='red', facecolor='none'
    )
    ax.add_patch(rect)
    
    ax.set_title('License Plate Detection Result')
    ax.axis('off')
    plt.show()
    
    return result

# Example usage:
# result = predict_license_plate('your_image.jpg')
# visualize_prediction('your_image.jpg')
'''
    
    inference_path = f"{model_path}_inference_example.py"
    with open(inference_path, 'w') as f:
        f.write(inference_code)
    print(f"   ✅ Inference example: {inference_path}")
    
    # 8. Create download package
    package_path = f"{model_path}_package.zip"
    with zipfile.ZipFile(package_path, 'w') as zipf:
        zipf.write(keras_model_path, f"{model_name}.keras")
        zipf.write(h5_model_path, f"{model_name}.h5")
        zipf.write(weights_path, f"{model_name}.weights.h5")
        zipf.write(architecture_path, f"{model_name}_architecture.json")
        zipf.write(metadata_path, f"{model_name}_metadata.json")
        zipf.write(inference_path, f"{model_name}_inference_example.py")
        
        # Add comprehensive README
        readme_content = f'''# License Plate Bounding Box Detection Model

## 🎯 Model Information
- **Name**: {model_name}
- **Type**: Bounding Box Regression CNN
- **Task**: License Plate Detection (Coordinate Prediction)
- **Created**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
- **TensorFlow Version**: {tf.__version__}

## 📦 Files Included
- `{model_name}.h5` - Complete model (recommended)
- `{model_name}_weights.h5` - Model weights only
- `{model_name}_architecture.json` - Model architecture
- `{model_name}_metadata.json` - Training metadata
- `{model_name}_inference_example.py` - Ready-to-use inference code

## 🚀 Quick Start
```python
import tensorflow as tf

# Load model
model = tf.keras.models.load_model('{model_name}.h5')

# Predict bounding box for an image
image = preprocess_your_image()  # Shape: (1, {IMG_SIZE}, {IMG_SIZE}, 3)
prediction = model.predict(image)  # Returns: [x_center, y_center, width, height]
```

## 📊 Model Performance
{json.dumps(training_results, indent=2)}

## 🔧 Model Specifications
- **Input**: {IMG_SIZE}x{IMG_SIZE}x3 RGB images (normalized 0-1)
- **Output**: 4 values [x_center, y_center, width, height] (normalized 0-1)
- **Parameters**: {model.count_params():,}
- **Loss Function**: Mean Squared Error (MSE)
- **Metrics**: IoU (Intersection over Union), MAE

## 📏 Coordinate Format
- All coordinates are normalized to [0, 1] range
- x_center, y_center: Center of bounding box
- width, height: Size of bounding box
- To convert to pixels: multiply by image dimensions

## 🎯 Use Cases
- License plate detection in traffic cameras
- Automatic license plate recognition (ALPR) systems
- Vehicle monitoring and tracking
- Parking management systems

## ⚡ Performance Tips
- Input images should be well-lit and clear
- Works best with frontal view license plates
- Consider image preprocessing for better results
- Use confidence thresholding based on your use case
'''
